# 5. GradientBoosting 실습

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
os.getcwd()

'/Users/jeonghyeonjeong/for github/머신러닝_데이터분석A-Z_패스트캠퍼스/8. 앙상블 기법의 종류와 원리'

### 1. 데이터 불러오기 및 전처리

In [5]:
data = pd.read_csv('/Users/jeonghyeonjeong/for github/머신러닝_데이터분석_dataset/otto_train.csv')
data.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [7]:
#의미 없다고 판단되는 변수 제거
data = data.drop(['id'], axis = 1)

In [8]:
#['target']변수의 문자열을 숫자로 변경
mapping_dict= { "Class_1" : 1, 
               "Class_2" : 2, 
               "Class_3" : 3, 
               "Class_4" : 4, 
               "Class_5" : 5, 
               "Class_6" : 6, 
               "Class_7" : 7, 
               "Class_8" : 8, 
               "Class_9" : 9
              }

after_mapping_target = data['target'].apply(lambda x: mapping_dict[x])

In [14]:
#설명 변수와 타겟 변수를 분리, 학습 데이터와 평가 데이터 분리
feature_columns = list(data.columns.difference(['target']))
X = data[feature_columns]
y = after_mapping_target

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(49502, 93) (12376, 93) (49502,) (12376,)


# 5-1. XGboost

## XGboost 설치 필요
terminal -> conda install -c conda-forge xgboost 

## XGboost 사용법 - xgboost in python 구글링
https://xgboost.readthedocs.io/en/latest/python/python_intro.html

https://xgboost.readthedocs.io/en/latest/parameter.html

In [22]:
import xgboost as xgb
import time
from sklearn.metrics import accuracy_score

#시작시간 지정
start = time.time()

#학습 데이터를 XGboost 모형에 맞게 변형
xgb_data_train = xgb.DMatrix(data = train_x, label = train_y)
#평가 데이터를 XGboost 모형에 맞게 변형
xgb_data_test = xgb.DMatrix(data = test_x)

#XGboost 모형에 필요한 parameter 지정
xgb_parameters = {'max_depth': 10, #트리 깊이
                'learning_rate' : 0.01, #step size
                 'n_estimate': 100, #트리 생성 개수
                 'objective': 'multi:softmax', #목적 함수
                 'num_class': len(set(train_y))+1 #분류 항목수
                 }


#학습 진행
xgb_model = xgb.train(params = xgb_parameters, dtrain = xgb_data_train)

#평가 데이터 예측
xgb_model_predict = xgb_model.predict(xgb_data_test)

#정확도 평가
print("Accuracy : %2f" %(accuracy_score(test_y, xgb_model_predict)*100), "%")

#코드 실행 시간 계산
print("Time : %2f" %(time.time() -start), "seconds")

Accuracy : 76.672592 %
Time : 31.652970 seconds


# 5-2. Lightgbm
## Lightgbm 설치 필요
terminal -> conda install -c conda-forge lightgbm  

## Lightgbm 사용법 - Lightgbm in python 구글링
https://lightgbm.readthedocs.io/en/latest/Python-Intro.html

https://lightgbm.readthedocs.io/en/latest/Parameters.html


In [27]:
import lightgbm as lgb

#시작시간 지정
start = time.time()

#학습 데이터를 LightGBM 모형에 맞게 변형
lgb_data_train = lgb.Dataset(data = train_x, label = train_y)
#평가 데이터를 LightGBM 모형에 맞게 변형
#lgb_data_test = lgb.Dataset(data = test_x) -> 이렇게 하면 나중에 lgb_model_predict = np.argmax(lgb_model.predict(lgb_test_data), axis = 1) 쓰면 dataset쓰지말고 raw data 쓰라고 에러남
#lgb_model_predict = np.argmax(lgb_model.predict(test_x), axis = 1) 그래서 그냥 변형 필요없이 이 단계에서 test_x), axis = 1이렇게 써주면 됨. 


#LightGBM 모형에 필요한 parameter 지정
lgb_parameters = {'max_depth': 10, #트리 깊이
                'learning_rate' : 0.01, #step size
                 'n_estimate': 100, #트리 생성 개수
                 'objective': 'multiclass', #목적 함수
                 'num_class': len(set(train_y))+1 #분류 항목수
                 }




In [32]:
#학습 진행
lgb_model = lgb.train(params = lgb_parameters, train_set = lgb_data_train)

#평가 데이터 예측 #softmax 결과 값 중 가장 최대인 것의 label로 예측
lgb_model_predict = np.argmax(lgb_model.predict(test_x), axis = 1)

#정확도 평가
print("Accuracy : %.2f" %(accuracy_score(test_y, lgb_model_predict)*100), "%")

#코드 실행 시간 계산
print("Time : %.2f" %(time.time() -start), "seconds")

Accuracy : 73.57 %
Time : 164.19 seconds


# 5-3. Catboost
## Catboost 설치 필요
terminal -> conda install -c conda-forge catboost 
## Catboost 사용법 - Catboost in python 구글링
https://catboost.ai/docs/concepts/python-reference_apply_catboost_model.html

https://catboost.ai/docs/concepts/parameter-tuning.html

In [41]:
import catboost as cb

#시작시간 지정
start = time.time()

#학습 데이터를 Catboost 모형에 맞게 변형
cb_data_train = cb.Pool(data = train_x, label = train_y)


#Catboost 모형에 필요한 parameter 지정
cb_parameters = {'max_depth': 10, #트리 깊이
                'learning_rate' : 0.01, #step size
                 'n_estimators': 100, #트리 생성 개수
                 'eval_metric': 'Accuracy',#평가 척도 지정
                 'loss_function': 'MultiClass' #목적 함수
                 }

In [43]:
#학습 진행
cb_model = cb.train(pool = cb_data_train, params = cb_parameters)

#평가 데이터 예측 #softmax 결과 값 중 가장 최대인 것의 label로 예측
cb_model_predict = np.argmax(cb_model.predict(test_x), axis = 1)

#정확도 평가
print("Accuracy : %.2f" %(accuracy_score(test_y, cb_model_predict)*100), "%")

#코드 실행 시간 계산
print("Time : %.2f" %(time.time() -start), "seconds")

0:	learn: 0.5907034	total: 699ms	remaining: 1m 9s
1:	learn: 0.6356107	total: 1.26s	remaining: 1m 1s
2:	learn: 0.6411256	total: 1.82s	remaining: 59s
3:	learn: 0.6480344	total: 2.38s	remaining: 57.1s
4:	learn: 0.6508222	total: 3.07s	remaining: 58.4s
5:	learn: 0.6499939	total: 3.75s	remaining: 58.8s
6:	learn: 0.6507818	total: 4.46s	remaining: 59.3s
7:	learn: 0.6548422	total: 5.18s	remaining: 59.6s
8:	learn: 0.6559533	total: 5.89s	remaining: 59.6s
9:	learn: 0.6560947	total: 6.61s	remaining: 59.5s
10:	learn: 0.6568421	total: 7.3s	remaining: 59.1s
11:	learn: 0.6588219	total: 7.84s	remaining: 57.5s
12:	learn: 0.6592259	total: 8.38s	remaining: 56.1s
13:	learn: 0.6611248	total: 8.91s	remaining: 54.8s
14:	learn: 0.6625591	total: 9.51s	remaining: 53.9s
15:	learn: 0.6631853	total: 10.1s	remaining: 53.1s
16:	learn: 0.6639328	total: 10.8s	remaining: 52.6s
17:	learn: 0.6668821	total: 11.4s	remaining: 52.1s
18:	learn: 0.6669630	total: 12.1s	remaining: 51.5s
19:	learn: 0.6675286	total: 13.5s	remaining: